In [ ]:
import pandas as pd
import numpy as np
import time, datetime, calendar
from colorcet import kbc
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize
import skyfield
from skyfield.api import EarthSatellite,load
hv.extension('bokeh')

In [ ]:
!du -h ../tle2017.csv

In [ ]:
tle = pd.read_csv('../tle2017.csv')

In [ ]:
print('Example TLE record:\n%r' % tle.iloc[0]['tle'])

In [ ]:

def compute_lon_lat(line1,line2):
    satellite = EarthSatellite(line1, line2, name='Unknown')
    geometry = satellite.at(satellite.epoch)

    subpoint = geometry.subpoint()
    if np.isnan(subpoint.latitude.degrees):
        return None
    if np.isnan(subpoint.longitude.degrees):
        return None
        
    return subpoint.longitude.degrees, subpoint.latitude.degrees

        

def lat_lon_from_lines(lines):
    lons, lats = [], []
    for line1, line2 in lines:
        lon, lat = compute_lon_lat(line1,line2)
        if None not in [lon, lat]:
            lons.append(lon)
            lats.append(lat)
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(np.array(lons), np.array(lats))
    return list(zip(eastings, northings))


In [ ]:
new_lines = [el.replace('None\n', '').split('\n')[:2] for el in tle['tle']] # Splitting the file

In [ ]:
coords = lat_lon_from_lines(new_lines) # Can take a while to run...

In [ ]:
hv.element.tiles.ESRI().opts(alpha=0.5, bgcolor='black') * rasterize(hv.Points(coords)).opts(width=900, height=600, cmap=kbc[64:], cnorm='eq_hist')

Loading the `export_test.h5` file into TLE lines (probably safe to ignore):

```python
import h5py
f = h5py.File('../vault/export_test.h5', 'r')
dataset = f['readout']
hdf5_lines = [(bytes(dataset[ind][2]).decode('ASCII').replace('\\', ''), 
  bytes(dataset[ind][3]).decode('ASCII').replace('\\', '')) for ind in range(100000)]
```